# TuLiP Example: Animal Herding

In [ ]:
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
import networkx as nx

from tulip import transys, spec, synth, dumpsmach, gridworld

## Construct a gridworld for the robot

In [ ]:
W = 7   # with of gridworld
H = 7   # height of gridworld

fts = transys.FTS()

# add nodes (state set X)
fts.add_nodes_from('a{}x{}'.format(i,j) for i in range(W) for j in range(H))

# add actions (input set U)
fts.sys_actions.add_from(['north', 'west', 'south', 'east'])

# add transitions
for i in range(W):
    for j in range(H):
        fts.add_edge( 'a{}x{}'.format(i,j), 'a{}x{}'.format(i,min(H-1,j+1)), sys_actions='north')
        fts.add_edge( 'a{}x{}'.format(i,j), 'a{}x{}'.format(i,max(0,  j-1)), sys_actions='south')
        fts.add_edge( 'a{}x{}'.format(i,j), 'a{}x{}'.format(min(W-1, i+1),j), sys_actions='east')
        fts.add_edge( 'a{}x{}'.format(i,j), 'a{}x{}'.format(max(0,   i-1),j), sys_actions='west')

# draw with networkx
pos = dict()
for i in range(W):
    for j in range(H):
        pos['a{}x{}'.format(i,j)] = (i,j)
        
nx.draw_networkx(fts, with_labels=True, arrows=True, node_size=500, pos = pos)

In [ ]:
# draw with tulip
fts.plot()

## Define atomic propositions and specification

In [ ]:
# Add atomic propositions
fts.atomic_propositions |= {'blocked', 'cat_target', 'mouse_target', 'dog_target'}

# define blocked states
blocked_list = ['a0x1', 'a0x6', 'a1x4', 'a2x1', 'a2x3', 'a3x3', \
                'a3x6', 'a4x0', 'a4x5', 'a4x6', 'a5x5', 'a5x6', \
                'a6x3', 'a6x6']
    
for state in blocked_list:
    fts.node[state]['ap'] |= {'blocked'}

fts.node['a6x0']['ap'] |= {'cat_target'}
fts.node['a6x5']['ap'] |= {'mouse_target'}
fts.node['a0x0']['ap'] |= {'dog_target'}

for state in fts.states():
    if state not in blocked_list:
        fts.states.initial.add(state)


# Formula variables
env_vars = {'dog', 'cat', 'mouse'}
sys_vars = {'bone', 'yarn', 'flute'}

# Intitialize and requirements
env_safe = set()
sys_safe = set()
sys_prog = set()

# Don't visit blocked regions
sys_safe |= {'! blocked'}

# Add some targets to visit
sys_prog |= {'eloc = "a0x2"', 'eloc = "a4x4"', 'eloc = "a5x0"', 'eloc = "a3x0"'}


#### TRANSLATE REQUIREMENTS TO SPECS ####

# REQ: If sensing an animal, herd at least one

# REQ: If no dog, dont show bone

# REQ: If no mouse, dont show flute

# REQ: If no cat, dont show yarn

# REQ: If herding, eventually reach target

# REQ: Can't herd dog and cat at the same time 

# REQ: Can't herd cat and mouse at the same time
    
# EXTRA: prioritize mouse over cat

formula = spec.GRSpec(env_vars=env_vars, 
                      sys_vars=sys_vars,
                      env_safety=env_safe,
                      sys_safety=sys_safe,
                      sys_prog=sys_prog)

print(formula.pretty())

## Solve synthesis problem

In [ ]:
# Set formula parameters
formula.moore = False     # can see next env state
formula.qinit = '\A \E'   # solve s.t. for all (\A) env initial conditions there exists (\E) sys initial condition 

fts.owner = 'env'

# Print complete spec (sys + spec)
# print synth._spec_plus_sys(formula, fts, None, False, False).pretty()

ctrl = synth.synthesize('omega', formula, env=fts)

dumpsmach.write_python_case("herding_policy.py", ctrl, classname="HerdingPolicy")

## Simulation

In [ ]:
from herding_policy import HerdingPolicy
from simulate_herding import simulate

h = HerdingPolicy()
simulate(h, fts)